# Best city to open a coffee shop - Toronto or New York?

In [18]:
import numpy as np

import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json



from geopy.geocoders import Nominatim

import requests
from pandas.io.json import json_normalize


import matplotlib.cm as cm
import matplotlib.colors as colors


from sklearn.cluster import KMeans


import folium
from folium import plugins
from folium.plugins import MarkerCluster


print('Libraries imported.')

Libraries imported.


In [19]:
conda install lxml

Solving environment: done

# All requested packages already installed.


Note: you may need to restart the kernel to use updated packages.


## 1. Toronto

### 1.1 Explore neighbourhood

In [20]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
table = pd.read_html(url)
table[0].head()

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [21]:
table[0].replace("Not assigned", np.nan, inplace=True)
table[0].head()

,Postal Code,Borough,Neighbourhood
0,M1A,NaN,NaN
1,M2A,NaN,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [22]:
table[0].dropna(subset=["Borough"], axis=0, inplace=True)
table[0].head(10)

,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
8,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
9,M1B,Scarborough,"Malvern, Rouge"
11,M3B,North York,Don Mills
12,M4B,East York,"Parkview Hill, Woodbine Gardens"
13,M5B,Downtown Toronto,"Garden District, Ryerson"


In [23]:
table[0].reset_index(drop=True, inplace=True)
table[0].head(10)

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


In [24]:
!pip install geocoder

import geocoder
from geopy.geocoders import Nominatim

In [25]:
lat_lng_coords = None

while(lat_lng_coords is None):
  g = geocoder.arcgis('{}, Toronto, Ontario'.format('Postal Code'))
  lat_lng_coords = g.latlng

latitude = lat_lng_coords[0]
longitude = lat_lng_coords[1]

print(latitude,longitude )

43.648690000000045 -79.38543999999996


In [26]:
url = 'http://cocl.us/Geospatial_data'
geotable = pd.read_csv(url)
geotable.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [27]:
toronto = pd.merge(table[0], geotable, left_on='Postal Code', right_on='Postal Code', left_index=False, right_index=False)
toronto.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


In [28]:
toronto.shape

(103, 5)

In [29]:
print('The dataframe has {} boroughs'.format(len(toronto['Borough'].unique())))

The dataframe has 10 boroughs


### 1.2 Explore and cluster the neighborhoods in Toronto

In [30]:
address = 'Toronto, Canada'

geolocator = Nominatim(user_agent="trt_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [32]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

for lat, lng, borough, neighbourhood in zip(toronto['Latitude'], toronto['Longitude'], toronto['Borough'], toronto['Neighbourhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

### 1.3 Explore Coffee Shops in the Toronto

Use Foursquare to explore the neighbourhood and segment them.

In [36]:
CLIENT_ID = 'YWCV0SJEXHKLIGKGFTWJMZMCQFK5TINABCDZUR4QVBUKC50K' # your Foursquare ID
CLIENT_SECRET = 'CBUZ1C0SIHYSWQMMKKPLMSGKUW1NB0BLLPQKHOV1QF4WB01N' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: YWCV0SJEXHKLIGKGFTWJMZMCQFK5TINABCDZUR4QVBUKC50K
CLIENT_SECRET:CBUZ1C0SIHYSWQMMKKPLMSGKUW1NB0BLLPQKHOV1QF4WB01N


Get the top 100 venues that are in Toronto within a radius of 500 meters.

In [58]:
LIMIT - 100
radius = 500

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    latitude, 
    longitude, 
    radius, 
    LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=YWCV0SJEXHKLIGKGFTWJMZMCQFK5TINABCDZUR4QVBUKC50K&client_secret=CBUZ1C0SIHYSWQMMKKPLMSGKUW1NB0BLLPQKHOV1QF4WB01N&v=20180605&ll=43.6563221,-79.3809161&radius=500&limit=100'

In [59]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5ff4c8358f29fd1b440ba058'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Bay Street Corridor',
  'headerFullLocation': 'Bay Street Corridor, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 96,
  'suggestedBounds': {'ne': {'lat': 43.6608221045, 'lng': -79.37470788695488},
   'sw': {'lat': 43.651822095499995, 'lng': -79.3871243130451}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '57eda381498ebe0e6ef40972',
       'name': 'UNIQLO ユニクロ',
       'location': {'address': '220 Yonge St',
        'crossStreet': 'at Dundas St W',
        'lat': 43.65591027779457,
        'lng': -79.38064099181345,
        'labeledLatLngs'

To sort category of venues.

In [60]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [61]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', ('venue.categories'), 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  This is separate from the ipykernel package so we can avoid doing imports until


,name,categories,lat,lng
0,UNIQLO ユニクロ,Clothing Store,43.655910,-79.380641
1,Ed Mirvish Theatre,Theater,43.655102,-79.379768
2,Silver Snail Comics,Comic Shop,43.657031,-79.381403
3,Yonge-Dundas Square,Plaza,43.656054,-79.380495
4,Blaze Pizza,Pizza Place,43.656518,-79.380015


In [62]:
nearby_venues.groupby('categories').count()

,name,lat,lng
categories,,,
Art Gallery,1,1,1
Art Museum,1,1,1
Bank,1,1,1
Bar,2,2,2
Bookstore,1,1,1
Bubble Tea Shop,2,2,2
Burger Joint,2,2,2
Burrito Place,1,1,1
Café,3,3,3


In [81]:
toronto_coffee_shops=nearby_venues[nearby_venues['categories'].str.contains('Café|Coffee Shop')]
toronto_coffee_shops

,name,categories,lat,lng
18,Hailed Coffee,Coffee Shop,43.658833,-79.383684
21,Tokyo Smoke,Coffee Shop,43.657230,-79.380870
30,Page One Cafe,Café,43.657772,-79.376073
33,Jimmy's Coffee,Coffee Shop,43.658421,-79.385613
36,Oakham Café,Café,43.658078,-79.378315
37,Tim Hortons,Coffee Shop,43.658570,-79.385123
48,Starbucks,Coffee Shop,43.654465,-79.378919
56,Balzac's Coffee,Coffee Shop,43.657854,-79.379200
66,Starbucks,Coffee Shop,43.659509,-79.382132
70,Second Cup,Café,43.652981,-79.377408


In [83]:
print('There are {} coffee shops in the Top 100 Venues in Toronto returned by Foursquare.'.format(toronto_coffee_shops.shape[0]))

There are 16 coffee shops in the Top 100 Venues in Toronto returned by Foursquare.


In [145]:
toronto_bookstore=nearby_venues[nearby_venues['categories'].str.contains('Bookstore')]
toronto_bookstore

,name,categories,lat,lng
19,Indigo,Bookstore,43.653515,-79.380696


In [146]:
print('There are {} bookstore in the Top 100 Venues in Toronto returned by Foursquare.'.format(toronto_bookstore.shape[0]))

There are 1 bookstore in the Top 100 Venues in Toronto returned by Foursquare.


In [108]:
map_toronto_coffee_shops = folium.Map(location=[latitude, longitude], zoom_start=10)

for lat, lng, name, categories in zip(toronto_coffee_shops['lat'], toronto_coffee_shops['lng'], toronto_coffee_shops['name'], toronto_coffee_shops['categories']):
    label = '{}, {}'.format(name, categories)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto_coffee_shops)  
    
map_toronto_coffee_shops

## 2. New York

### 1.1 Explore neighbourhood

In [109]:
!wget -q -O 'newyork_data.json' https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/labs/newyork_data.json
print('Data downloaded!')

Data downloaded!


In [110]:
with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)

In [111]:
newyork_data

{'type': 'FeatureCollection',
 'totalFeatures': 306,
 'features': [{'type': 'Feature',
   'id': 'nyu_2451_34572.1',
   'geometry': {'type': 'Point',
    'coordinates': [-73.84720052054902, 40.89470517661]},
   'geometry_name': 'geom',
   'properties': {'name': 'Wakefield',
    'stacked': 1,
    'annoline1': 'Wakefield',
    'annoline2': None,
    'annoline3': None,
    'annoangle': 0.0,
    'borough': 'Bronx',
    'bbox': [-73.84720052054902,
     40.89470517661,
     -73.84720052054902,
     40.89470517661]}},
  {'type': 'Feature',
   'id': 'nyu_2451_34572.2',
   'geometry': {'type': 'Point',
    'coordinates': [-73.82993910812398, 40.87429419303012]},
   'geometry_name': 'geom',
   'properties': {'name': 'Co-op City',
    'stacked': 2,
    'annoline1': 'Co-op',
    'annoline2': 'City',
    'annoline3': None,
    'annoangle': 0.0,
    'borough': 'Bronx',
    'bbox': [-73.82993910812398,
     40.87429419303012,
     -73.82993910812398,
     40.87429419303012]}},
  {'type': 'Feature',
 

In [112]:
neighborhoods_data = newyork_data['features']

In [113]:
neighborhoods_data[0]

{'type': 'Feature',
 'id': 'nyu_2451_34572.1',
 'geometry': {'type': 'Point',
  'coordinates': [-73.84720052054902, 40.89470517661]},
 'geometry_name': 'geom',
 'properties': {'name': 'Wakefield',
  'stacked': 1,
  'annoline1': 'Wakefield',
  'annoline2': None,
  'annoline3': None,
  'annoangle': 0.0,
  'borough': 'Bronx',
  'bbox': [-73.84720052054902,
   40.89470517661,
   -73.84720052054902,
   40.89470517661]}}

In [122]:
# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
newyork = pd.DataFrame(columns=column_names)

In [123]:
newyork

,Borough,Neighborhood,Latitude,Longitude


In [124]:
for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    newyork = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

In [125]:
newyork.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


In [129]:
address = 'New York City, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
ny_latitude = location.latitude
ny_longitude = location.longitude
print('The geograpical coordinate of New York City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of New York City are 40.7127281, -74.0060152.


In [130]:
# create map of New York using latitude and longitude values
map_newyork = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(newyork['Latitude'], newyork['Longitude'], newyork['Borough'], newyork['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  
    
map_newyork

1.2 Explore and cluster the neighborhoods in Toronto

In [128]:
CLIENT_ID = 'YWCV0SJEXHKLIGKGFTWJMZMCQFK5TINABCDZUR4QVBUKC50K' # your Foursquare ID
CLIENT_SECRET = 'CBUZ1C0SIHYSWQMMKKPLMSGKUW1NB0BLLPQKHOV1QF4WB01N' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: YWCV0SJEXHKLIGKGFTWJMZMCQFK5TINABCDZUR4QVBUKC50K
CLIENT_SECRET:CBUZ1C0SIHYSWQMMKKPLMSGKUW1NB0BLLPQKHOV1QF4WB01N


In [131]:
LIMIT-100
radius = 500

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    ny_latitude, 
    ny_longitude, 
    radius, 
    LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=YWCV0SJEXHKLIGKGFTWJMZMCQFK5TINABCDZUR4QVBUKC50K&client_secret=CBUZ1C0SIHYSWQMMKKPLMSGKUW1NB0BLLPQKHOV1QF4WB01N&v=20180605&ll=40.7127281,-74.0060152&radius=500&limit=100'

In [132]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5ff4d73d69128074fb7cce16'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'},
    {'name': '$-$$$$', 'key': 'price'}]},
  'headerLocation': 'Downtown Manhattan',
  'headerFullLocation': 'Downtown Manhattan, New York',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 108,
  'suggestedBounds': {'ne': {'lat': 40.7172281045, 'lng': -74.00008952063419},
   'sw': {'lat': 40.7082280955, 'lng': -74.0119408793658}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '57f0689d498e7d49d9189369',
       'name': 'The Bar Room at Temple Court',
       'location': {'address': '123 Nassau St',
        'lat': 40.7114477287544,
        'lng': -74.00680157032005,
        'labe

In [133]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [134]:
venues = results['response']['groups'][0]['items']
    
nearby_venues_2 = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues_2 =nearby_venues_2.loc[:, filtered_columns]

# filter the category for each row
nearby_venues_2['venue.categories'] = nearby_venues_2.apply(get_category_type, axis=1)

# clean columns
nearby_venues_2.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues_2.head()

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  This is separate from the ipykernel package so we can avoid doing imports until


,name,categories,lat,lng
0,The Bar Room at Temple Court,Hotel Bar,40.711448,-74.006802
1,Alba Dry Cleaner & Tailor,Laundry Service,40.711434,-74.006272
2,"The Beekman, A Thompson Hotel",Hotel,40.711173,-74.006702
3,City Hall Park,Park,40.711893,-74.007792
4,Gibney Dance Center Downtown,Dance Studio,40.713923,-74.005661


In [135]:
print('{} venues were returned by Foursquare.'.format(nearby_venues_2.shape[0]))

100 venues were returned by Foursquare.


In [136]:
nearby_venues_2.groupby('categories').count()

,name,lat,lng
categories,,,
American Restaurant,2,2,2
Antique Shop,1,1,1
Asian Restaurant,1,1,1
Baby Store,1,1,1
Bagel Shop,1,1,1
Bakery,1,1,1
Bookstore,1,1,1
Boxing Gym,1,1,1
Breakfast Spot,1,1,1


In [137]:
nyc_coffee_shops=nearby_venues_2[nearby_venues_2['categories'].str.contains('Café|Coffee Shop')]
nyc_coffee_shops

,name,categories,lat,lng
5,The Wooly Daily,Coffee Shop,40.712137,-74.008395
16,Birch Coffee,Coffee Shop,40.710759,-74.005953
27,Pisillo Italian Cafe,Café,40.710493,-74.007546
34,Konditori,Café,40.709474,-74.006630
61,Suited,Coffee Shop,40.709402,-74.008098
62,Voyager Espresso,Coffee Shop,40.708787,-74.007063
66,Hungry Ghost,Coffee Shop,40.715379,-74.007476


In [138]:
print('There are {} coffee shops in the Top 100 Venues in New York returned by Foursquare.'.format(nyc_coffee_shops.shape[0]))

There are 7 coffee shops in the Top 100 Venues in New York returned by Foursquare.


In [142]:
nyc_bookstore=nearby_venues_2[nearby_venues_2['categories'].str.contains('Bookstore')]
nyc_bookstore

,name,categories,lat,lng
37,The Mysterious Bookshop,Bookstore,40.71491,-74.00948


In [143]:
print('There are {} bookstore in the Top 100 Venues in New York returned by Foursquare.'.format(nyc_bookstore.shape[0]))

There are 1 bookstore in the Top 100 Venues in New York returned by Foursquare.


In [139]:
map_nyc_coffee_shops = folium.Map(location=[latitude, longitude], zoom_start=10)

for lat, lng, name, categories in zip(nyc_coffee_shops['lat'], nyc_coffee_shops['lng'], nyc_coffee_shops['name'], nyc_coffee_shops['categories']):
    label = '{}, {}'.format(name, categories)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_nyc_coffee_shops)  
    
map_nyc_coffee_shops